In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import json

In [11]:
PATH = '/media/popo/Elements/Datasets/Leaf-disease-classification/'

In [14]:
label_dict = json.load(open(f'{PATH}label_num_to_disease_map.json'))
train_ids = pd.read_csv(f'{PATH}train.csv')

In [15]:
train_ids

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3
...,...,...
21392,999068805.jpg,3
21393,999329392.jpg,3
21394,999474432.jpg,1
21395,999616605.jpg,4
